In [ ]:
import requests
from pymongo import MongoClient
from bson.objectid import ObjectId
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib2
import sys